In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from transformers import pipeline
import tensorflow

from ast import literal_eval

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-04 10:55:21.881491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 268M/268M [00:16<00:00, 16.2MB/s] 
2022-12-04 10:55:54.203343: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was tra

In [6]:
data = ["I love you", "I am running a marathon without shoes"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9845187664031982},
 {'label': 'NEGATIVE', 'score': 0.9984996318817139}]

In [33]:
df_billboard = pd.read_csv('data/sentiment_selection.csv')
df_billboard.head(5)

,song,artist,url,lyrics,lyrics_text,rank,peak_rank,weeks_on_board,last_week,genre,...,Unnamed: 1678,Unnamed: 1679,Unnamed: 1680,Unnamed: 1681,Unnamed: 1682,Unnamed: 1683,Unnamed: 1684,Unnamed: 1685,Unnamed: 1686,Unnamed: 1687
0,Physical,Olivia Newton-John,https://genius.com/Olivia-newton-john-physical...,"[('[Verse 1]', ""i'm sayin' all the things that...",i'm sayin' all the things that i know you'll l...,"{'1982-03-27': 95, '1982-03-20': 56, '1982-03-...","{'1982-03-27': 1, '1982-03-20': 1, '1982-03-13...","{'1982-03-27': 26, '1982-03-20': 25, '1982-03-...","{'1982-03-27': 56.0, '1982-03-20': 46.0, '1982...","['adult standards', 'australian dance', 'disco...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Let Love Rule,Lenny Kravitz,https://genius.com/Lenny-kravitz-let-love-rule...,"[('[Verse 1]', ""love is gentle as a rose and l...",love is gentle as a rose and love can conquer ...,"{'1990-02-17': 99, '1990-02-10': 91, '1990-02-...","{'1990-02-17': 89, '1990-02-10': 89, '1990-02-...","{'1990-02-17': 4, '1990-02-10': 3, '1990-02-03...","{'1990-02-17': 91.0, '1990-02-10': 89.0, '1990...","['permanent wave', 'pop rock', 'rock']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,On And On And On,ABBA,https://genius.com/Abba-on-and-on-and-on-lyrics,"[('[Verse 1]', 'i was at a party and this fell...","i was at a party and this fella said to me ""so...","{'1981-08-01': 99, '1981-07-25': 94, '1981-07-...","{'1981-08-01': 90, '1981-07-25': 90, '1981-07-...","{'1981-08-01': 6, '1981-07-25': 5, '1981-07-18...","{'1981-08-01': 94.0, '1981-07-25': 95.0, '1981...","['europop', 'swedish pop']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I Need Love,LL Cool J,https://genius.com/Ll-cool-j-i-need-love-lyrics,"[('[Verse 1]', 'when i\'m alone in my room som...",when i'm alone in my room sometimes i stare at...,"{'1987-10-24': 98, '1987-10-17': 60, '1987-10-...","{'1987-10-24': 14, '1987-10-17': 14, '1987-10-...","{'1987-10-24': 13, '1987-10-17': 12, '1987-10-...","{'1987-10-24': 60.0, '1987-10-17': 46.0, '1987...","['east coast hip hop', 'golden age hip hop', '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Straight Outta Compton,N.W.A,https://genius.com/Nwa-straight-outta-compton-...,"[('[Intro: Dr. Dre]', 'you are now about to wi...",you are now about to witness the strength of s...,"{'2015-09-12': 61, '2015-09-05': 38}","{'2015-09-12': 38, '2015-09-05': 38}","{'2015-09-12': 2, '2015-09-05': 1}","{'2015-09-12': 38.0, '2015-09-05': nan}","['conscious hip hop', 'gangster rap', 'golden ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# df_billboard['lyrics'] = df_billboard['lyrics'].map(literal_eval)
df_billboard = df_billboard[df_billboard['lyrics_text'].notnull()]

In [46]:
# lyrics = df_billboard['lyrics'].to_list()
# lyrics[0]
splitter = 10

def process_lyrics(lyrics):
    if lyrics:
        lyrics_list = []
        lyrics_split = lyrics.split(' ')
        num_words = len(lyrics_split)
        counter = num_words // splitter
        for i in range(10):
            if i == 9:
                lyrics_curr = lyrics_split[(i * counter) - counter // 4:]
            elif i == 0:
                lyrics_curr = lyrics_split[(i * counter):((i + 1) * counter + counter // 4)]
            else:
                lyrics_curr = lyrics_split[(i * counter)  - counter // 4 : ((i + 1) * counter + counter // 4)]
            lyrics_curr = " ".join(lyrics_curr)
            lyrics_list.append(lyrics_curr)
    else:
        print(lyrics)
    return lyrics_list

In [47]:
lyrics_split_by_10 = df_billboard['lyrics_text'].map(process_lyrics)

In [48]:
lyrics_split_by_10[33]

["if i die young, bury me in satin lay me down on a bed of roses sink me in the river at dawn send me away with the words of a love song oh-oh, oh-oh lord, make me a rainbow, i'll shine down",
 "the words of a love song oh-oh, oh-oh lord, make me a rainbow, i'll shine down on my mother she'll know i'm safe with you when she stands under my colors oh, and life ain't always what you think it ought to be, no ain't even gray, but she buries her",
 "always what you think it ought to be, no ain't even gray, but she buries her baby the sharp knife of a short life well, i've had just enough time if i die young, bury me in satin lay me down on a bed of roses sink me in the river",
 "me in satin lay me down on a bed of roses sink me in the river at dawn send me away with the words of a love song the sharp knife of a short life well, i've had just enough time and i'll be wearing white when i come into your",
 "well, i've had just enough time and i'll be wearing white when i come into your kingdom

In [49]:
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

def get_positivity_scores(text):
    raw_scores = sentiment_pipeline(text)
    scores = [raw_score['score'] if raw_score['label'] == 'POSITIVE' else -raw_score['score'] for raw_score in raw_scores]
    return scores


def append_positivity(all_lyrics):
    positivity_list = []
    for lyrics in all_lyrics:
        # parts_text = list(zip(*lyrics))
        if lyrics:
            positivity = get_positivity_scores(lyrics)
            positivity_list.append(positivity)
        else:
            positivity_list.append([])
    df_billboard['positivity'] = positivity_list

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceCla

In [50]:
append_positivity(lyrics_split_by_10)

In [51]:
df_billboard.to_csv('data/positivity_on_selection.csv')